In [1]:
import math

from flask import render_template, url_for

from imgurpython import ImgurClient#上傳圖片程序
import time

#畫圖程序
import matplotlib.pyplot as plt
import numpy as np
#畫圖程序

import ctypes
import pandas
import requests
from bs4 import BeautifulSoup

from datetime import datetime

from flask import Flask, request, abort

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError,LineBotApiError
)
from linebot.models import (
    LeaveEvent,SourceGroup,JoinEvent,SourceUser,FollowEvent,ImageCarouselColumn,ImageCarouselTemplate,CarouselColumn,CarouselTemplate,ImageMessage,MessageEvent, TextMessage, TextSendMessage,LocationMessage,FollowEvent,PostbackEvent,ImageSendMessage,FollowEvent,TemplateSendMessage,ButtonsTemplate,PostbackTemplateAction,MessageTemplateAction,URITemplateAction
)

#圖片辨識流程by face++
import requests
import pandas
import sqlite3 as lite
from json import JSONDecoder
#圖片辨識流程by face++

#import cv2
import os,shutil#資料移動相關
import random
import configparser as ConfigParser

ini_data=ConfigParser.ConfigParser()
ini_data.read('key.ini')

#line資訊
line_bot_api = LineBotApi(ini_data.get('line_bot','key'))
handler = WebhookHandler(ini_data.get('line_bot','secret'))
#line資訊

#facepp資訊
key = ini_data.get('facepp','key')
secret = ini_data.get('facepp','secret')
#facepp資訊

In [ ]:
app = Flask(__name__)

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'



@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print(event.message.text)
        
    #使用者要求寫入資料庫
    av_data={'文字':[event.message.text],'時間':[time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())]}
    write=pandas.DataFrame(av_data)

    with lite.connect('av_data.sqlite') as db:
        write.to_sql('user_say',con=db,if_exists='append')
    #使用者要求寫入資料庫
        
    if event.message.text=='眾天使':
        
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from face_search',con = db)
            
        word='\n'.join(list(set(read['名子'].tolist())))
        av_amount=str(len(list(set(read['名子'].tolist()))))
        
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='目前天使庫：'+av_amount+'位\n'+word))
        
    elif event.message.text=='許願':
        
        #隨機選擇一位老婆

        three_boss=list(range(1,2+1))
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from face_search',con = db)
            select=list(set(read['名子'].tolist()))

            for item in three_boss:
                select+=['馬英九']
                select+=['蔡英文']
                select+=['柯p']

            rad=random.randint(0,len(select)-1)
            sent_name=read[read['名子']==select[rad]].iloc[0]['名子']
            sent_logo=read[read['名子']==select[rad]].iloc[0]['logo']
            if sent_logo=='':
                sent_logo='https://i.imgur.com/JrohLrx.jpg'
            
            if sent_name=='馬英九' or sent_name=='蔡英文' or sent_name=='柯p':
                set_label='你好 我是三幻神之一'
                set_title='墮天使'
                set_text='神抽'
            elif sent_name=='紗倉真菜' or sent_name=='三上悠亞' or sent_name=='高橋聖子':
                set_label='你好 我是三大神之一'
                set_title='主天使'
                set_text='神抽'
                
            elif sent_name=='光頭葛格':
                set_label='你好 我是最大雷'
                set_title='露西法'
                set_text='降臨'
            else:
                set_label='你好 我是'
                set_title='天使'
                set_text='降臨'
            
            sent_Column=CarouselColumn(
                thumbnail_image_url=sent_logo,
                title=set_title,
                text=set_text,
                actions=[
                    PostbackTemplateAction(
                        label=set_label,
                        text=' ',
                        data='action=buy&itemid=1'
                    ),
                    MessageTemplateAction(
                        label=sent_name,
                        text=' '
                    ),
                    URITemplateAction(
                        label='按這搜尋去～',
                        uri='https://www.google.com.tw/search?q='+sent_name
                    )
                ]
            )


        #隨機選擇一位老婆

        carousel_template_message = TemplateSendMessage(
            alt_text='Carousel template',
            template=CarouselTemplate(
                columns=[sent_Column
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token,carousel_template_message)
        
    elif event.message.text=='建立' and isinstance(event.source, SourceGroup):
        print(event.source.user_id)
        by_group_id=event.source.group_id

        av_data={'group_id':[by_group_id],'接受建立':['y'],'模式':['KUSO']}
        write=pandas.DataFrame(av_data)
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from select_group_id',con = db)
            if sum(read['group_id']==by_group_id)==0:
                with lite.connect('av_data.sqlite') as db:
                    write.to_sql('select_group_id',con=db,if_exists='append')
            else:
                with lite.connect('av_data.sqlite') as db:
                    db.cursor().execute('UPDATE select_group_id set 接受建立 = "y" where group_id="'+by_group_id+'"')

        line_bot_api.reply_message(
                    event.reply_token,
                    TextSendMessage(text='知道了!請傳送圖片 如果要取消建立 請輸入"取消"'))
        
        
        
#######################開發者模式##############################     
        
    elif event.message.text[0:event.message.text.find('\n')]=='建立女優' and event.message.text.count('\n')==3 :
        
        process_word=event.message.text
        
        cli_name=process_word[process_word.find('\n')+1:len(process_word)][0:process_word[process_word.find('\n')+1:len(process_word)].find('\n')]
        cli_av_file_name=process_word[process_word.find('\n')+1:len(process_word)][process_word[process_word.find('\n')+1:len(process_word)].find('\n')+1:len(process_word[process_word.find('\n')+1:len(process_word)])][0:process_word[process_word.find('\n')+1:len(process_word)][process_word[process_word.find('\n')+1:len(process_word)].find('\n')+1:len(process_word[process_word.find('\n')+1:len(process_word)])].find('\n')]
        cli_web=process_word[process_word.find('\n')+1:len(process_word)][process_word[process_word.find('\n')+1:len(process_word)].find('\n')+1:len(process_word[process_word.find('\n')+1:len(process_word)])][process_word[process_word.find('\n')+1:len(process_word)][process_word[process_word.find('\n')+1:len(process_word)].find('\n')+1:len(process_word[process_word.find('\n')+1:len(process_word)])].find('\n')+1:len(process_word[process_word.find('\n')+1:len(process_word)][process_word[process_word.find('\n')+1:len(process_word)].find('\n')+1:len(process_word[process_word.find('\n')+1:len(process_word)])])]
        
        #建立女優資料##################################
        key = "B2LK45DwbvhBhz6NFcja5q4m3182NMmk"
        secret = "h3jMXHC-fc0aXdcuo_cQOQBERuICfZHA"

        name=[cli_name]#人物名子
        ps=['']#人物附註
        av_file_name=cli_av_file_name+'.jpg'#人物檔名 一定要羅馬拼音

        #製作女優資料夾
        try:
            os.mkdir('av_girl_data/'+name[0])
        except FileExistsError as e:
            None
        #製作女優資料夾

        #製作女優圖片
        response=requests.get(cli_web,stream=True)
        print(response)

        with open(av_file_name,'wb') as f:
            f.write(response.content)
        #製作女優圖片

        #製作女優圖片檔名排序
        dir_file=os.listdir('av_girl_data/'+name[0])
        file_name=[]
        for item in dir_file:
            file_name+=[int(item[item.find('_')+1:item.find('.jpg')])]
        if file_name==[]:
            file_name=[0]
        #製作女優圖片檔名排序

        #移動圖片
        file_end='av_girl_data/'+name[0]+'/'+av_file_name[0:av_file_name.find('.jpg')]+'_'+str(max(file_name)+1)+'.jpg'
        shutil.move(av_file_name,file_end)
        #移動圖片

        ##########################################################################

        #製作facetoken######################先上傳判斷############################
        http_url = "https://api-cn.faceplusplus.com/facepp/v3/detect"

        decide=1#try跳開
        count=0#失敗計數數量
        while decide:
            time.sleep(1)
            try:
                with  open(file_end, "rb") as f:
                    data = {"api_key": key, "api_secret": secret}
                    files = {"image_file": f}
                    response = requests.post(http_url, data=data, files=files)

                    req_con = response.content.decode('utf-8')
                    req_dict = JSONDecoder().decode(req_con)

                    print(req_dict)
                    print('人臉數量'+str(len(req_dict['faces'])))
                    decide=0
            except KeyError as e3:
                count=count+1
                print('field')
                if count==8:
                    decide=0
        #######################先上傳判斷############################


        #######################將剪裁上傳############################
        if len(req_dict['faces'])==1:#判斷圖片是否只有一人
            left=req_dict['faces'][0]['face_rectangle']['left']
            top=req_dict['faces'][0]['face_rectangle']['top']
            width=req_dict['faces'][0]['face_rectangle']['width']
            height=req_dict['faces'][0]['face_rectangle']['height']

            tri=10#方框加回來

            #讓opencv可以讀中文路徑
            def cv_imread(filePath):  
                cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)  
                ## imdecode读取的是rgb，如果后续需要opencv处理的话，需要转换成bgr，转换后图片颜色会变化  
                ##cv_img=cv2.cvtColor(cv_img,cv2.COLOR_RGB2BGR)  
                return cv_img  
            #讓opencv可以讀中文路徑

            #讀取
            image = cv_imread(file_end)
            #剪裁 因為錯誤剪取所以取消
            #cv2.imencode('.jpg',image[top-tri: top+height+tri,left-tri:left+width+tri])[1].tofile(file_end)  
            #剪裁後讀取 因為錯誤剪取所以取消
            #image = cv_imread(file_end)

            decide=1#try跳開
            count=0#失敗計數數量
            while decide:
                time.sleep(1)
                try:
                    data = {"api_key": key, "api_secret": secret}
                    files = {"image_file": open(file_end, "rb")}
                    response = requests.post(http_url, data=data, files=files)

                    req_con = response.content.decode('utf-8')
                    req_dict = JSONDecoder().decode(req_con)

                    print(req_dict)
                    print('人臉數量'+str(len(req_dict['faces'])))
                    decide=0
                except KeyError as e3:
                    count=count+1
                    print('field')
                    if count==8:
                        decide=0
            ##製作facetoken######################將剪裁上傳############################

            #紀錄資料庫程序
            pinyin=[av_file_name[0:av_file_name.find('.jpg')]]
            face_token=[req_dict['faces'][0]['face_token']]
            
            #logo檢查程序 如果有就增加 沒有就空白
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from face_search',con = db)
                if sum(read["名子"]==name[0])!=0:
                    logo=read[read["名子"]==name[0]].iloc[0,6]
                else:
                    logo=''
            #logo檢查程序 如果有就增加 沒有就空白

            av_data={'名子':name,'羅馬拼音':pinyin,'face_tokens':face_token,'附註':ps,'檔名':[av_file_name[0:av_file_name.find('.jpg')]+'_'+str(max(file_name)+1)+'.jpg'],'logo':[logo]}
            write=pandas.DataFrame(av_data)

            print('=======================================')

            #在faceset裡面新增臉
            http_url = "https://api-cn.faceplusplus.com/facepp/v3/faceset/addface"

            decide=1#try跳開
            count=0#失敗計數數量
            while decide:
                time.sleep(1)
                try:
                    data = {"api_key": key, "api_secret": secret, "outer_id": "line_bot_av_girl","face_tokens":face_token[0]}

                    print('確認'+face_token[0])

                    response = requests.post(http_url, data=data)

                    req_con = response.content.decode('utf-8')
                    req_dict = JSONDecoder().decode(req_con)

                    print(response)
                    print(response.text)

                    #寫入資料程序(成功上傳再寫入)
                    with lite.connect('av_data.sqlite') as db:
                        write.to_sql('face_search',con=db,if_exists='append')

                    with lite.connect('av_data.sqlite') as db:
                        read=pandas.read_sql_query('select * from face_search',con = db)
                        print('網路人臉數量:'+str(eval(response.text)["face_count"]))
                        print('伺服器人臉數量:'+str(len(read['檔名'].tolist())))

                    decide=0
                except KeyError as e3:
                    count=count+1
                    print('field')
                    if count==8:
                        decide=0
                        
            #電腦顯示存檔位置
            print(file_end)
        
            line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='已進入開發者模式新建女優'))
        else:
            #電腦告知錯誤人數
            print('圖片人數超過一人')
            os.remove(file_end)
            print('已刪除')
            line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='已進入開發者模式 但圖片人數超過一人'))
        #建立女優資料##################################
        
#######################開發者模式##############################  

    elif event.message.text=='切換模式':
        if isinstance(event.source, SourceGroup):
            print(event.source.group_id)
            by_group_id=event.source.group_id
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from KUSO_mode',con = db)
            if sum(read['id']==by_group_id)==0:
                av_data={'id':[by_group_id],'模式':['KUSO']}
                write=pandas.DataFrame(av_data)
                with lite.connect('av_data.sqlite') as db:
                    write.to_sql('KUSO_mode',con=db,if_exists='append')
                line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='已進入KUSO模式'))
            else:
                with lite.connect('av_data.sqlite') as db:
                    db.execute('DELETE FROM KUSO_mode WHERE id="'+by_group_id+'"')
                line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='已進入AV模式'))


        if isinstance(event.source, SourceUser):
            print(event.source.user_id)
            by_user_id=event.source.user_id
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from KUSO_mode',con = db)
            if sum(read['id']==by_user_id)==0:
                av_data={'id':[by_user_id],'模式':['KUSO']}
                write=pandas.DataFrame(av_data)
                with lite.connect('av_data.sqlite') as db:
                    write.to_sql('KUSO_mode',con=db,if_exists='append')
                line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='已進入KUSO模式'))
            else:
                with lite.connect('av_data.sqlite') as db:
                    db.execute('DELETE FROM KUSO_mode WHERE id="'+by_user_id+'"')
                line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='已進入AV模式'))



    elif event.message.text[0:2]=='取消':
        
        if isinstance(event.source, SourceGroup):
            print(event.source.group_id)
            by_group_id=event.source.group_id
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_group_id',con = db)
                if sum(read['group_id']==by_group_id)!=0:
                    with lite.connect('av_data.sqlite') as db:
                        db.execute('DELETE FROM select_group_id WHERE group_id="'+by_group_id+'"')
                        line_bot_api.reply_message(
                            event.reply_token,
                            TextSendMessage(text='已取消'))
    
            
        elif isinstance(event.source, SourceUser):
            print(event.source.user_id)
            by_user_id=event.source.user_id
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_user_id',con = db)
                if sum(read['user_id']==by_user_id)!=0:
                    with lite.connect('av_data.sqlite') as db:
                        db.execute('DELETE FROM select_user_id WHERE user_id="'+by_user_id+'"')
                        line_bot_api.reply_message(
                            event.reply_token,
                            TextSendMessage(text='已取消'))
            
        




    elif event.message.text[0:4]=='建立天使' and isinstance(event.source, SourceUser):
        print(event.source.user_id)
        by_user_id=event.source.user_id

        if event.message.text[4:len(event.message.text)]=='':
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='你建立後面沒打附註壓~'))

        else:
            av_data={'user_id':[by_user_id],'接受建立':['y'],'模式':['AV'],'名子跟附註':[event.message.text[4:len(event.message.text)]]}
            write=pandas.DataFrame(av_data)
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_user_id',con = db)
                if sum(read['user_id']==by_user_id)==0:
                    with lite.connect('av_data.sqlite') as db:
                        write.to_sql('select_user_id',con=db,if_exists='append')
                else:
                    with lite.connect('av_data.sqlite') as db:
                        db.cursor().execute('UPDATE select_user_id set 接受建立 = "y" where user_id="'+by_user_id+'"')


            line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='知道了!請傳送圖片 如果要取消建立 請輸入"取消"'))
        
    elif event.message.text[0:6]=='建立KUSO' and isinstance(event.source, SourceUser):
        print(event.source.user_id)
        by_user_id=event.source.user_id

        if event.message.text[6:len(event.message.text)]=='':
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='你建立後面沒打附註壓~'))

        else:
            av_data={'user_id':[by_user_id],'接受建立':['y'],'模式':['KUSO'],'名子跟附註':[event.message.text[6:len(event.message.text)]]}
            write=pandas.DataFrame(av_data)
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_user_id',con = db)
                if sum(read['user_id']==by_user_id)==0:
                    with lite.connect('av_data.sqlite') as db:
                        write.to_sql('select_user_id',con=db,if_exists='append')
                else:
                    with lite.connect('av_data.sqlite') as db:
                        db.cursor().execute('UPDATE select_user_id set 接受建立 = "y" where user_id="'+by_user_id+'"')


            line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='知道了!請傳送圖片 如果要取消建立 請輸入"取消"'))

    
    
    elif event.message.text[0:4]=='建立天使' and isinstance(event.source, SourceGroup):
        print(event.source.group_id)
        by_group_id=event.source.group_id

        if event.message.text[4:len(event.message.text)]=='':
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='你建立後面沒打附註壓~'))

        else:
            av_data={'group_id':[by_group_id],'接受建立':['y'],'模式':['AV'],'名子跟附註':[event.message.text[4:len(event.message.text)]]}
            write=pandas.DataFrame(av_data)
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_group_id',con = db)
                if sum(read['group_id']==by_group_id)==0:
                    with lite.connect('av_data.sqlite') as db:
                        write.to_sql('select_group_id',con=db,if_exists='append')
                else:
                    with lite.connect('av_data.sqlite') as db:
                        db.cursor().execute('UPDATE select_group_id set 接受建立 = "y" where group_id="'+by_group_id+'"')


            line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='知道了!請傳送圖片 如果要取消建立 請輸入"取消"'))

    elif event.message.text[0:6]=='建立KUSO' and isinstance(event.source, SourceGroup):
        print(event.source.group_id)
        by_group_id=event.source.group_id

        if event.message.text[6:len(event.message.text)]=='':
            line_bot_api.reply_message(event.reply_token,TextSendMessage(text='你建立後面沒打附註壓~'))

        else:
            av_data={'group_id':[by_group_id],'接受建立':['y'],'模式':['KUSO'],'名子跟附註':[event.message.text[6:len(event.message.text)]]}
            write=pandas.DataFrame(av_data)
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_group_id',con = db)
                if sum(read['group_id']==by_group_id)==0:
                    with lite.connect('av_data.sqlite') as db:
                        write.to_sql('select_group_id',con=db,if_exists='append')
                else:
                    with lite.connect('av_data.sqlite') as db:
                        db.cursor().execute('UPDATE select_group_id set 接受建立 = "y" where group_id="'+by_group_id+'"')


            line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='知道了!請傳送圖片 如果要取消建立 請輸入"取消"'))

    elif event.message.text=='使用說明':
        print(event.message.text)
        
        #使用者要求寫入資料庫
        av_data={'文字':[event.message.text],'時間':[time.strftime("%Y-%m-%d %H:%M:%S",time.localtime())]}
        write=pandas.DataFrame(av_data)
        
        with lite.connect('av_data.sqlite') as db:
            write.to_sql('user_say',con=db,if_exists='append')
        #使用者要求寫入資料庫
        
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text='你好!請對我們傳送圖片~\n\n我們來幫你找你的天使w\n\n對了 你可以拍照截圖做到以下幾點 天使會更容易找到：\n１．正面臉\n２．清晰照\n３．不截到其他人頭\n\n另外 你可以\n輸入"許願"：來找尋天使\n輸入"建立天使"：並在後面打入附註 來建立智庫'))
    

    
    else:
        None


        
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    
    def image_search(mode):
        #圖片儲存流程
        message_content = line_bot_api.get_message_content(event.message.id)

        with open('test.jpg', 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)
        #圖片儲存流程


        decide=1#重複比對照片
        test=0#失敗5次以上說看不懂
        while decide:
            time.sleep(0.5)#讓程式停止一秒 避免face++伺服器當掉
            try:
                key = "B2LK45DwbvhBhz6NFcja5q4m3182NMmk"
                secret = "h3jMXHC-fc0aXdcuo_cQOQBERuICfZHA"

                #想要的face數量
                want_face=3

                #在faceset搜索
                http_url = "https://api-cn.faceplusplus.com/facepp/v3/search"

                filepath = "test.jpg"

                data = {"api_key": key, "api_secret": secret, "outer_id": mode,"return_result_count":want_face}
                files = {"image_file": open(filepath, "rb")}
                response = requests.post(http_url, data=data, files=files)

                req_con = response.content.decode('utf-8')
                req_dict = JSONDecoder().decode(req_con)
                print(req_dict)


                #與資料庫名子匹配
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from face_search',con = db)
                ans_name=read[read["face_tokens"]==req_dict['results'][0]['face_token']].iloc[0,2]    
                ans_ps=read[read["face_tokens"]==req_dict['results'][0]['face_token']].iloc[0,5]
                ans_av_file_name=read[read["face_tokens"]==req_dict['results'][0]['face_token']].iloc[0,3]
                ans_logo=read[read["face_tokens"]==req_dict['results'][0]['face_token']].iloc[0,6]


                #與資料庫名子匹配
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from face_search',con = db)
                ans_name2=read[read["face_tokens"]==req_dict['results'][1]['face_token']].iloc[0,2]    
                ans_ps2=read[read["face_tokens"]==req_dict['results'][1]['face_token']].iloc[0,5]
                ans_av_file_name2=read[read["face_tokens"]==req_dict['results'][1]['face_token']].iloc[0,3]
                ans_logo2=read[read["face_tokens"]==req_dict['results'][1]['face_token']].iloc[0,6]


                #與資料庫名子匹配
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from face_search',con = db)
                ans_name3=read[read["face_tokens"]==req_dict['results'][2]['face_token']].iloc[0,2]    
                ans_ps3=read[read["face_tokens"]==req_dict['results'][2]['face_token']].iloc[0,5]
                ans_av_file_name3=read[read["face_tokens"]==req_dict['results'][2]['face_token']].iloc[0,3]
                ans_logo3=read[read["face_tokens"]==req_dict['results'][2]['face_token']].iloc[0,6]


                if ans_ps=='':
                    ans_ps='好好照顧她!知道嗎'
                if ans_ps2=='':
                    ans_ps2='好好照顧她!知道嗎'
                if ans_ps3=='':
                    ans_ps3='好好照顧她!知道嗎'

                #人正確
                if req_dict['results'][0]['confidence']>=81.0:  

                    if ans_logo=='':#如果沒有logo統一用此網址
                        ans_logo='https://i.imgur.com/JrohLrx.jpg'
                    if ans_logo2=='':#如果沒有logo統一用此網址
                        ans_logo2='https://i.imgur.com/JrohLrx.jpg'
                    if ans_logo3=='':#如果沒有logo統一用此網址
                        ans_logo3='https://i.imgur.com/JrohLrx.jpg'

                    carousel_template_message = TemplateSendMessage(
                        alt_text='Carousel template',
                        template=CarouselTemplate(
                            columns=[
                                CarouselColumn(
                                    thumbnail_image_url=ans_logo,
                                    title='相似度：',
                                    text=str(req_dict['results'][0]['confidence'])[0:str(req_dict['results'][0]['confidence']).find('.')]+'趴',
                                    actions=[
                                        PostbackTemplateAction(
                                            label='我認為是',
                                            text=' ',
                                            data='action=buy&itemid=1'
                                        ),
                                        MessageTemplateAction(
                                            label=ans_name,
                                            text=' '
                                        ),
                                        URITemplateAction(
                                            label='按這搜尋去～',
                                            uri='https://www.google.com.tw/search?q='+ans_name
                                        )
                                    ]
                                )                            
                            ]
                        )
                    )
                    line_bot_api.reply_message(event.reply_token,carousel_template_message)
                    #移動傳過來的檔案搜尋數
                    shutil.copy2('test.jpg','av_girl_data/sure/'+ans_name+'_'+str(req_dict['results'][0]['confidence'])+'_'+str(time.time())+'.jpg')

                    #人臉搜尋數辨識排行
                    with lite.connect('av_data.sqlite') as db:
                        read=pandas.read_sql_query('select * from charts',con = db)

                    if sum((read['名子']==ans_name).tolist())==0:
                        av_data={'名子':[ans_name],'搜尋數':[0]}
                        write=pandas.DataFrame(av_data)
                        with lite.connect('av_data.sqlite') as db:
                            write.to_sql('charts',con=db,if_exists='append')

                    with lite.connect('av_data.sqlite') as db:
                        read=pandas.read_sql_query('select * from charts',con = db)

                    girl_number=read[read['名子']==ans_name]['搜尋數'].tolist()[0]
                    for item in sorted(read['搜尋數'].tolist()):
                        if girl_number==item:
                            girl_number=girl_number+1
                        #print(girl_number)
                    with lite.connect('av_data.sqlite') as db:
                        db.cursor().execute('UPDATE charts set 搜尋數 = "'+str(girl_number)+'" where 名子="'+ans_name+'"')
                    
                    #人臉辨識排行

                #人不確定 傳送三個 記得label只能有12個字
                elif 38.0<req_dict['results'][0]['confidence'] and req_dict['results'][0]['confidence']<81.0:

                    if ans_logo=='':#如果沒有logo統一用此網址
                        ans_logo='https://i.imgur.com/Vq9uNlE.jpg'
                    if ans_logo2=='':#如果沒有logo統一用此網址
                        ans_logo2='https://i.imgur.com/Vq9uNlE.jpg'
                    if ans_logo3=='':#如果沒有logo統一用此網址
                        ans_logo3='https://i.imgur.com/Vq9uNlE.jpg'

                    name_first=CarouselColumn(
                            thumbnail_image_url=ans_logo,
                            title='相似度：',
                            text=str(req_dict['results'][0]['confidence'])[0:str(req_dict['results'][0]['confidence']).find('.')]+'趴',
                            actions=[
                                PostbackTemplateAction(
                                    label='我有點看不清壓 但應該是',
                                    text=' ',
                                    data='action=buy&itemid=1'
                                ),
                                MessageTemplateAction(
                                    label=ans_name,
                                    text=' '
                                ),
                                URITemplateAction(
                                    label='按這搜尋去～',
                                    uri='https://www.google.com.tw/search?q='+ans_name
                                )
                            ]
                        )

                    name_second=CarouselColumn(
                            thumbnail_image_url=ans_logo2,
                            title='相似度：',
                            text=str(req_dict['results'][1]['confidence'])[0:str(req_dict['results'][1]['confidence']).find('.')]+'趴',
                            actions=[
                                PostbackTemplateAction(
                                    label='我有點看不清壓 但應該是',
                                    text=' ',
                                    data='action=buy&itemid=2'
                                ),
                                MessageTemplateAction(
                                    label=ans_name2,
                                    text=' '
                                ),
                                URITemplateAction(
                                    label='按這搜尋去～',
                                    uri='https://www.google.com.tw/search?q='+ans_name2
                                )
                            ]
                        )

                    name_third=CarouselColumn(
                            thumbnail_image_url=ans_logo3,
                            title='相似度：',
                            text=str(req_dict['results'][2]['confidence'])[0:str(req_dict['results'][2]['confidence']).find('.')]+'趴',
                            actions=[
                                PostbackTemplateAction(
                                    label='我有點看不清壓 但應該是',
                                    text=' ',
                                    data='action=buy&itemid=3'
                                ),
                                MessageTemplateAction(
                                    label=ans_name3,
                                    text=' '
                                ),
                                URITemplateAction(
                                    label='按這搜尋去～',
                                    uri='https://www.google.com.tw/search?q='+ans_name3
                                )
                            ]
                        )

                    columns=[name_first,name_second,name_third]

                    if ans_name==ans_name2:
                        columns.remove(name_second)

                    if ans_name==ans_name3:
                        columns.remove(name_third)

                    if ans_name2==ans_name3 and len(columns)==3:
                        columns.remove(name_third)

                    carousel_template_message = TemplateSendMessage(
                        alt_text='Carousel template',
                        template=CarouselTemplate(columns
                        )
                    )

                    line_bot_api.reply_message(event.reply_token,carousel_template_message)
                    #移動傳過來的檔案
                    shutil.copy2('test.jpg','av_girl_data/not_sure/'+ans_name+'_'+str(req_dict['results'][0]['confidence'])+'_'+str(time.time())+'.jpg')

                    #人臉辨識排行
                    '''
                    with lite.connect('av_data.sqlite') as db:
                        read=pandas.read_sql_query('select * from charts',con = db)

                    if sum((read['名子']==ans_name).tolist())==0:
                        av_data={'名子':[ans_name],'搜尋數':[0]}
                        write=pandas.DataFrame(av_data)
                        with lite.connect('av_data.sqlite') as db:
                            write.to_sql('charts',con=db,if_exists='append')

                    with lite.connect('av_data.sqlite') as db:
                        read=pandas.read_sql_query('select * from charts',con = db)

                    girl_number=read[read['名子']==ans_name]['搜尋數'].tolist()[0]
                    for item in sorted(read['搜尋數'].tolist()):
                        if girl_number==item:
                            girl_number=girl_number+1
                        #print(girl_number)
                    with lite.connect('av_data.sqlite') as db:
                        db.cursor().execute('UPDATE charts set 搜尋數 = "'+str(girl_number)+'" where 名子="'+ans_name+'"')
                    '''
                    #人臉辨識排行

                else:
                    line_bot_api.reply_message(
                        event.reply_token,
                        TextSendMessage(text='找不到像的人啊...'))

                decide=0
            except KeyError as e:
                print('filed')
                test=test+1
                if test==5:
                    print('無法解析')
                    if isinstance(event.source, SourceUser):
                        line_bot_api.reply_message(
                            event.reply_token,
                            TextSendMessage(text='找不到像的人啊...'))
                    decide=0
    
    
    if isinstance(event.source, SourceGroup):
        print(event.source.group_id)
        by_group_id=event.source.group_id
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from select_group_id',con = db)

        if sum(read['group_id']==by_group_id)!=0:
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_group_id',con = db)
                ps=read[read['group_id']==by_group_id].iloc[0,2].replace('\n',' ')
                ps=ps.replace('//','')
                ps=ps.replace(':','')
                mode=read[read['group_id']==by_group_id].iloc[0,4]
            
             #圖片儲存流程
            message_content = line_bot_api.get_message_content(event.message.id)

            with open('av_girl_data/Client/'+mode+'_'+ps+'_群組_'+by_group_id+'_'+time.strftime("%Y-%m-%d %H-%M-%S",time.localtime())+'.jpg', 'wb') as fd:
                for chunk in message_content.iter_content():
                    fd.write(chunk)
            #圖片儲存流程

            with lite.connect('av_data.sqlite') as db:
                db.execute('DELETE FROM select_group_id WHERE group_id="'+by_group_id+'"')

            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='已建立 我們將會審查您的圖片'))
        else:
            if isinstance(event.source, SourceGroup):
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from KUSO_mode',con = db)
                    if sum(read['id']==by_group_id)==0:
                        image_search("line_bot_av_girl")
                    else:
                        image_search("line_bot_av_KUSO")
            elif isinstance(event.source, SourceUser):
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from KUSO_mode',con = db)
                    if sum(read['id']==by_user_id)==0:
                        image_search("line_bot_av_girl")
                    else:
                        image_search("line_bot_av_KUSO")

    elif isinstance(event.source, SourceUser):
        print(event.source.user_id)
        by_user_id=event.source.user_id
        
        with lite.connect('av_data.sqlite') as db:
            read=pandas.read_sql_query('select * from select_user_id',con = db)
        if sum(read['user_id']==by_user_id)!=0:
            with lite.connect('av_data.sqlite') as db:
                read=pandas.read_sql_query('select * from select_user_id',con = db)
                ps=read[read['user_id']==by_user_id].iloc[0,2].replace('\n',' ')
                ps=ps.replace('//','')
                ps=ps.replace(':','')
                mode=read[read['user_id']==by_user_id].iloc[0,4]
            
             #圖片儲存流程
            message_content = line_bot_api.get_message_content(event.message.id)

            with open('av_girl_data/Client/'+mode+'_'+ps+'_使用者_'+by_user_id+'_'+time.strftime("%Y-%m-%d %H-%M-%S",time.localtime())+'.jpg', 'wb') as fd:
                for chunk in message_content.iter_content():
                    fd.write(chunk)
            #圖片儲存流程

            with lite.connect('av_data.sqlite') as db:
                db.execute('DELETE FROM select_user_id WHERE user_id="'+by_user_id+'"')

            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text='已建立 我們將會審查您的圖片'))
        else:
            if isinstance(event.source, SourceGroup):
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from KUSO_mode',con = db)
                    if sum(read['id']==by_group_id)==0:
                        image_search("line_bot_av_girl")
                    else:
                        image_search("line_bot_av_KUSO")
            elif isinstance(event.source, SourceUser):
                with lite.connect('av_data.sqlite') as db:
                    read=pandas.read_sql_query('select * from KUSO_mode',con = db)
                    if sum(read['id']==by_user_id)==0:
                        image_search("line_bot_av_girl")
                    else:
                        image_search("line_bot_av_KUSO")
    
    



if __name__ == "__main__":
    app.run(port=5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jun/2018 00:15:19] "GET /av_update HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2018 00:24:23] "GET /av_update HTTP/1.1" 200 -
